<a href="https://colab.research.google.com/github/txxshalott/Unlicensed-AI-therapist/blob/main/Unlicensed_AI_Therapist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI therapist prototype
Author: Shalott Tam\
Date: April 18 - May 2\
Description: Not robust... use at your own risk



In [ ]:
!pip install openai
!pip install python-dotenv

import openai
import os
from dotenv import load_dotenv
from google.colab import drive
drive.mount('//ColabNotebooks', force_remount=True)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
_dotenv_path = '//ColabNotebooks/Colab Notebooks/Unlicensed_therapist/.env'
load_dotenv(_dotenv_path)
openai.api_key = os.getenv("API_KEY")

None


In [ ]:
#@title The prototype { vertical-output: true }
#@markdown Advice isnt terrible but a little wonky at times, use at your own risk (or dont cause it costs money too lol get real therapy)

MAX_LEN = 200
SUMMARY_TOKENS = 200
response = ""
lastResponse = "" #previous response from therapist
summary = ""

def therapist(prompt):
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=70,
        n=1,
        stop=None,
        temperature=0.7, #higher = more creative
        frequency_penalty = 0.7, #phrase, higher = try not to generate similar response
    )
    message = completions.choices[0].text.strip()
    return message

def summarize(prompt): #summarize, leaving the last few lines
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt="Summarize the following conversation: " + prompt,
        max_tokens=SUMMARY_TOKENS,
        n=1,
        stop=None,
        temperature=0.4,
    )
    summary = completions.choices[0].text.strip()
    return summary

# Start convo
starting = "The following is a conversation with a helpful, wise, and friendly therapist. First, they help the client open up about their problems, after a while they start offering recommendations."
prompt = starting
print("Therapist: Hi there! I am your AI therapist, how are you feeling today?")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye", "goodbye"]: # exit
      print("Hope you feel better, bye!")
      break

    prompt += "\nClient: " + user_input

    promptLen = len(prompt.split()) # approx tokens
    if promptLen > MAX_LEN:
      summary = summarize(prompt)  # summarize convo and make it the prompt

      response = therapist(starting + summary + "\nTherapist: ")
    else:
      response = therapist(prompt + "\nTherapist: ")

    ### IGNORE SECTION
    # remove first sentence from response bc it always repeats what the user said
    # but it only happens for a short while edit: ?? doesnt happen anymore
    # if promptLen < 100:
    #   print("response before token: " + response)
    #   sentences = sent_tokenize(response)
    #   response = ' '.join(sentences[1:]) #slice 1 onward
    ###

    # if last sentence is not a complete sentence, ie last character it's not a punctuation
    doc = nlp(response)
    if doc[-1].is_alpha:
      response = response[:response.rfind(".")] + "." # remove cut off sentence (taken from bryson)

    lastResponse = "\nTherapist: " + response
    print(lastResponse)

    prompt += response


In [ ]:
# data processing
import numpy as np
import json

f = open("/content/drive/MyDrive/ColabNotebooks/intentsData.json")
data = json.load(f)
f.close()

with open("output.jsonl", "w") as f: # w means write mode
  for intent in data["intents"]:
      for pattern in intent["patterns"]: # each correspond to a response
          for response in intent["responses"]:
              prompt = pattern.strip()
              completion = response.strip()
              json.dump({"prompt": prompt, "completion": completion}, f)
              f.write('\n')

In [ ]:
# wrap output text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
#@title Fine-tuned model
#@markdown Trained with only a single data set, not robust and more expensive to run :[\ { display-mode: "form" }
#@markdown Tell the therapist how you're feeling below
user_input = "" #@param {type:"string"}
model_id = os.getenv("TUNED_MODEL_ID")
def thisSucksOmg(prompt):
    completions = openai.Completion.create(
        engine=model_id, #should be model_id after fine tuning
        prompt=prompt,
        max_tokens=20,
        n=1,
        stop=None,
        temperature=0.7,
        frequency_penalty = 0.5, #phrase, higher = try not to generate similar response
        presence_penalty = 0, #word
    )
    message = completions.choices[0].text.strip()
    return message

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye", "goodbye"]:
      print("Hope you feel better, bye!")
      break

    prompt += "\nClient: " + user_input
    response = therapist(prompt + "\nTherapist: ")

    lastResponse = "\nTherapist: " + response
    print(lastResponse)

    prompt += response

You may ignore everything below (failed attempts, debugging stuff)


In [ ]:
### could not tune model on colab, did it in terminal instead

def fine_tune_model(data):
    response = openai.FineTune.create(
        training_file=data,
        model="davinci",
        prompt_loss_weight=0.1,
        n_epochs=1,
        batch_size=1,
        suffix="out-of-tune"
    )

    model_id = response.model.id
    return model_id

prompt = "The following is a conversation with a helpful, creative, clever, and friendly therapist. At the beginning, they ask questions to learn more about the client's problems, emotional state, history. The goal is to help client open up about their problems."
model_id = fine_tune_model("file name or smth")
print(model_id)

In [ ]:
### runs fine tuned model

def runModel(modelName):
  gen_response = modelName
  prompt = "The following is a conversation with a helpful, creative, clever, and friendly therapist. At the beginning, they ask questions to learn more about the client's problems, emotional state, history. The goal is to help client open up about their problems."
  print("AI: Hi there! I am your AI therapist, how are you feeling today?")

  while True:
    user_input = input("User: ")

    if user_input.lower() in ["exit", "quit", "bye", "goodbye"]:
      print("Hope you feel better, bye!")
      break

    prompt += "\nUser: " + user_input
    response = gen_response(prompt + "\nTherapist: ")
    response = response[:response.rfind(".")] + "." # taken from bryson: remove cut off sentence

    prompt += "\nTherapist: " + response #therapist response
    print(response)

In [ ]:
### debug stuff

import pandas as pd

# Load the training data into a pandas DataFrame
df = pd.DataFrame(formatted_data)

# Check if the "prompt" column exists
if 'prompt' not in df.columns:
    print('Error: "prompt" column is missing in the training data')
else:
    print('Training data has a "prompt" column')